In [1]:
import os
import random
from datetime import datetime

now = datetime.now()

# Folder name to save registration transforms
resultsFolder = "results" + now.strftime("%Y_%m_%d_%H_%M_%S")
print("Results folder name:", resultsFolder)

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# random shuffle data
random.shuffle(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
atlases = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path

Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)

print("Saving training data names.")
file = open(f"./{resultsFolder}/trainData.txt", "w") 
for i, name in enumerate(trainData):
    print(f"\tSaving {name}. {i+1}/{len(data)}")
    file.write(name+"\n")
file.close()

print("Saving atlases names.")
file = open(f"./{resultsFolder}/atlases.txt", "w") 
for i, name in enumerate(atlases):
    print(f"\tSaving {name}. {i+1}/{len(data)}")
    file.write(name+"\n")
file.close()

Results folder name: results2020_10_03_21_45_09
Saving training data names.
	Saving 9006723.nii.gz. 1/46
	Saving 9090290.nii.gz. 2/46
	Saving 9065272.nii.gz. 3/46
	Saving 9041946.nii.gz. 4/46
	Saving 9075815.nii.gz. 5/46
	Saving 9056326.nii.gz. 6/46
	Saving 9003430.nii.gz. 7/46
	Saving 9093622.nii.gz. 8/46
	Saving 9047539.nii.gz. 9/46
	Saving 9002430.nii.gz. 10/46
	Saving 9017909.nii.gz. 11/46
	Saving 9004175.nii.gz. 12/46
	Saving 9001104.nii.gz. 13/46
	Saving 9031426.nii.gz. 14/46
	Saving 9011420.nii.gz. 15/46
	Saving 9057417.nii.gz. 16/46
	Saving 9023193.nii.gz. 17/46
	Saving 9002817.nii.gz. 18/46
	Saving 9034644.nii.gz. 19/46
	Saving 9012867.nii.gz. 20/46
	Saving 9074437.nii.gz. 21/46
	Saving 9036770.nii.gz. 22/46
	Saving 9083500.nii.gz. 23/46
	Saving 9005075.nii.gz. 24/46
	Saving 9011115.nii.gz. 25/46
	Saving 9080864.nii.gz. 26/46
	Saving 9013161.nii.gz. 27/46
	Saving 9081306.nii.gz. 28/46
	Saving 9040944.nii.gz. 29/46
	Saving 9044005.nii.gz. 30/46
	Saving 9036287.nii.gz. 31/46
	Sa

In [2]:
import SimpleITK as sitk
import numpy as np
from utils import *

transforms = []
mseList = []
P = [3, 3, 3]
N = [5, 5, 5]

# Read fixed image
pathName = os.path.join(path, atlases[0])
fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)


for i, f in enumerate(trainData[:1]): # for test
# for i, f in enumerate(trainData):
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    # Histogram matching
    image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                                   fixedImage)

    label = sitk.ReadImage(os.path.join(labelPath, f))

    t = registration(fixedImage, image, label)

    # add transform to the list
    transforms.append(t)

    # add mse to the list
    mseList.append(mse3D(fixedImage, resampleImage(image, fixedImage, t)))
    
# sort mseList
mseListSorted = sorted(enumerate(mseList), key=lambda x: x[1])

# pick the desired number of atlases
atlasesNum = 1
minx = miny = minz = 10000
maxx = maxy = maxz = 0
images = []
labels = []

###############################################################################
# Read atlases and labels and tranform them
###############################################################################
for i in range(atlasesNum):
    idx = mseListSorted[i][0]
    t = transforms[idx]
    f = trainData[idx]
    pathName = os.path.join(path, f)
    print(f'Reading {f}. {i+1}/{atlasesNum}')
    
    # Histogram matching
    images.append(sitk.GetArrayFromImage(resampleImage(
                sitk.HistogramMatching(
                    sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                       timeStep = 0.04, 
                                       numberOfIterations = 10),
                                    fixedImage), 
                  fixedImage, t)))
    
    label = sitk.ReadImage(os.path.join(labelPath, f))
    label = sitk.GetArrayFromImage(resampleSegmentation(label, fixedImage, t))
    labels.append(label)
    
    idxs = np.nonzero(label != 0)
    minx = min([minx, min(idxs[0])])
    maxx = max([maxx, max(idxs[0])])
    miny = min([miny, min(idxs[1])])
    maxy = max([maxy, max(idxs[1])])
    minz = min([minz, min(idxs[2])])
    maxz = max([maxz, max(idxs[2])])
    
    
###############################################################################
# Calculate the desired shape
###############################################################################
shape, copyShape, offset, length = calculateCropShape(images, P, N, minx, maxx, miny, maxy, minz, maxz)

###############################################################################
# crop images and labels
###############################################################################
for i in range(len(images)):
    newImage = np.zeros((shape[0][1] - shape[0][0] + 1, 
                         shape[1][1] - shape[1][0] + 1, 
                         shape[2][1] - shape[2][0] + 1
                        ), dtype="uint16", order="C")
    newImage[offset[0]:offset[0]+length[0], 
             offset[1]:offset[1]+length[1], 
             offset[2]:offset[2]+length[2]] = images[i][copyShape[0][0]:copyShape[0][1]+1,
                                                        copyShape[1][0]:copyShape[1][1]+1,
                                                        copyShape[2][0]:copyShape[2][1]+1,]
    images[i] = newImage

    newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, 
                          shape[1][1] - shape[1][0] + 1, 
                          shape[2][1] - shape[2][0] + 1,
                         ), dtype="uint8", order="C")
    newLabels[offset[0]:offset[0]+length[0], 
              offset[1]:offset[1]+length[1], 
              offset[2]:offset[2]+length[2]] = labels[i][copyShape[0][0]:copyShape[0][1]+1,
                                                         copyShape[1][0]:copyShape[1][1]+1,
                                                         copyShape[2][0]:copyShape[2][1]+1,]
    labels[i] = newLabels

fixedImage = sitk.GetArrayFromImage(fixedImage)
newImage = np.zeros((shape[0][1] - shape[0][0] + 1, 
                     shape[1][1] - shape[1][0] + 1, 
                     shape[2][1] - shape[2][0] + 1
                    ), dtype="uint16", order="C")
newImage[offset[0]:offset[0]+length[0], 
         offset[1]:offset[1]+length[1], 
         offset[2]:offset[2]+length[2]] = fixedImage[copyShape[0][0]:copyShape[0][1]+1,
                                                     copyShape[1][0]:copyShape[1][1]+1,
                                                     copyShape[2][0]:copyShape[2][1]+1,]
fixedImage = newImage

###############################################################################
# Make to the proper type
###############################################################################
print("Fixing data types.")
images = np.array(images, order='C')
labels = np.array(labels, order='C')
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

###############################################################################
# Perform segmentation
###############################################################################
from cGen import cGen
print("Running segmentation.")
'''
segmentation = cGen.applySPBM(fixedImage, images, labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)
'''
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRC(fixedImage, images, labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              # xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)
                              xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11)

Registration of 9006723.nii.gz. 1/32
Reading 9006723.nii.gz. 1/1
Fixing data types.
Running segmentation.
X: 10 	Time: 6.198883056640625e-06


In [ ]:
####################################################
# END OF NORMAL CODE 
####################################################

In [3]:
# fix error
saveSegmentation(segmentationSPBM,
                 os.path.join(path, atlases[0]), 
                 os.path.join(resultsFolder, atlases[0]) + "SPBM.mhd", 
                 copyShape, 
                 offset,
                 length)

ValueError: could not broadcast input array from shape (330,187,94) into shape (330,246,94)

In [ ]:
# Save segmentation
pathName = os.path.join(path, atlases[0])
originalImage = sitk.ReadImage(pathName)
originalImageShape = originalImage.GetSize()[::-1]
finalSegmentation = np.zeros(originalImageShape, dtype='uint8')
finalSegmentation[copyShape[0][0]:copyShape[0][1]+1,
                  copyShape[2][0]:copyShape[1][1]+1,
                  copyShape[2][0]:copyShape[2][1]+1,] = segmentation[offset[0]:offset[0]+length[0], 
                                                                     offset[1]:offset[1]+length[1], 
                                                                     offset[2]:offset[2]+length[2]]

finalSegmentation = sitk.GetImageFromArray(finalSegmentation)
finalSegmentation.CopyInformation(originalImage)
sitk.WriteImage(finalSegmentation, imageNames[0]+'rt.mhd')

In [ ]:
print(len(fixedImage))
print(len(images))
print(len(labels))
print(fixedImage.shape)
print(images.shape)
print(labels.shape)
print(fixedImage.flags)
print(images.flags)
print(labels.flags)

In [ ]:
help(images)

In [ ]:
'''
TESTS
'''

# sort mseList
mseListSorted = sorted(enumerate(mseList), key=lambda x: x[1])

# pick the desired number of atlases
atlasesNum = 5
minx = miny = minz = 10000
maxx = maxy = maxz = 0
images = []
labels = []

# Read atlases and labels and tranform them
for i in range(2):
    idx = mseListSorted[i][0]
    print(idx)
    t = transforms[idx]
    f = trainData[idx]
    pathName = os.path.join(path, f)
    
    # Histogram matching
    images.append(resampleImage(sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                                         fixedImage), 
                  fixedImage, t))
    
    label = sitk.ReadImage(os.path.join(labelPath, f))
    label = sitk.GetArrayFromImage(resampleSegmentation(label, fixedImage, t))
    
    idxs = np.nonzero(label != 0)
    minx = min([minx, min(idxs[0])])
    maxx = max([maxx, max(idxs[0])])
    miny = min([miny, min(idxs[1])])
    maxy = max([maxy, max(idxs[1])])
    minz = min([minz, min(idxs[2])])
    maxz = max([maxz, max(idxs[2])])

In [ ]:
from PIL import Image

def showImg_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(images[1], 60)

In [ ]:
l = sitk.GetArrayFromImage(label)
l[l == 4] = 0
mask = sitk.GetImageFromArray(l)

showImg4_(mask, 100)


In [ ]:
'''
labels
1: katw gonato
2: katw xondos
3: panw gonato
4: panw xondros
5: panw xondros
'''

In [ ]:
idx = 0
f = trainData[idx]
pathName = os.path.join(path, f)
image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                               fixedImage)
'''
label = sitk.ReadImage(os.path.join(labelPath, f))
# Create the mask
l = sitk.GetArrayFromImage(label)
l[l>1] = 1
mask = sitk.GetImageFromArray(l)
mask.CopyInformation(label)
'''
    
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(image)
# elastixImageFilter.AddMovingMask(mask)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("affine"))
elastixImageFilter.Execute()

In [ ]:
resultImage2 = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

In [ ]:
sitk.PrintParameterMap(elastixImageFilter.GetParameterMap())

In [ ]:
help(sitk.ElastixImageFilter)

In [ ]:
print(transformParameterMap)

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(resultImage, 60)

In [ ]:
showImg_(fixedImage, 60)
showImg_(images[1], 60)
showImg_(resultImage, 60)

In [ ]:
help(sitk.GetImageFromArray)